Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Tarea 5 </h2>
    <h1> Ensembles </h1>
    <p>
        Profesor Marcelo Mendoza<br>
        Primer Semestre 2023<br>    
        Fecha de entrega: Miércoles  8 de Junio
    </p>
    <br>
</center>

<br>

---

## Indicaciones

Deberás entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas. 

**IMPORTANTE**: 
- Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas. Es decir, si tienes un código perfecto pero este no es explicado o no se responden preguntas asociadas a este, no se tendrá el puntaje completo.
- El notebook debe tener todas las celdas de código ejecutadas. Cualquier notebook que no las tenga no podrá ser corregido.
- El carácter de esta tarea es **INDIVIDUAL**. Cualquier instancia de copia resultará en un 1,1 como nota de curso.

---

In [18]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

import numpy as np
import pandas as pd

## Parte 1: Carga de datos
En esta tarea debes usar el dataset de Titanic que puedes descargar desde [Kaggle](https://www.kaggle.com/competitions/titanic/data?select=train.csv), solo necesitas el archivo `train.csv`.

Describe el dataset.

In [19]:
df = pd.read_csv("train.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Parte 2: Preprocesamiento (15 ptos.)
Primero debes separar los datos en conjuntos de training y test (70/30).

Luego, es necesario preprocesar los datos, por lo que debes considerar si es necesario hacer entre otras, las siguientes cosas:
- Remover columnas
- Normalizar variables
- Manejo de valores nulos

En este caso se necesita hacer un paso de preprocesamiento particular. Para manejar los datos categoricos tienes que usar One Hot Encoding.

Recuerda justificar cada paso de preprocesamiento que hagas.

### Preguntas:
- ¿Por que es necesario usar One Hot Encoding?¿Cuál es el problema con usar numeros enteros para codificar las categorias?

In [20]:
# Chequear missing values
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [21]:
print(df[df['Embarked'].isnull()])

# Tanto Icard, Miss. Amelie como Stone, Mrs. George Nelson (Martha Evelyn) embarcaron en Southampton, por lo que corriguemos cambiando por una S
# https://www.encyclopedia-titanica.org/titanic-survivor/amelia-icard.html
# https://www.encyclopedia-titanica.org/titanic-survivor/martha-evelyn-stone.html

df['Embarked'] = df['Embarked'].fillna('S')


     PassengerId  Survived  Pclass                                       Name   
61            62         1       1                        Icard, Miss. Amelie  \
829          830         1       1  Stone, Mrs. George Nelson (Martha Evelyn)   

        Sex   Age  SibSp  Parch  Ticket  Fare Cabin Embarked  
61   female  38.0      0      0  113572  80.0   B28      NaN  
829  female  62.0      0      0  113572  80.0   B28      NaN  


In [22]:
# Arreglamos la columna Age, sacando la media de las edades y colocandolo en los valores nulos, de esta forma no perdemos datos y tenemos una prediccion mas precisa
age = df.Age.dropna()
age.mean().round(0)

df['Age'] = df['Age'].fillna(age.mean().round(0))

# Dropeamos las columnas que no aportan nada para la prediccion
df = df.drop(['Ticket'], axis=1)
df = df.drop(['Fare'], axis=1)
df = df.drop(['Name'], axis=1)
df = df.drop(['PassengerId'], axis=1)
df = df.drop(['SibSp'], axis=1)
df = df.drop(['Parch'], axis=1)

In [23]:
# Como no conocemos la cabina de todos los tripulantes, cambiaremos todos los valores nulos a M
# De la misma forma, cambiaremos la cabina a seccion, por ejemplo, si tenemos C85, quedaria en C

df['Cabin'] = df['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'M')

In [24]:
df

,Survived,Pclass,Sex,Age,Cabin,Embarked
0,0,3,male,22.0,M,S
1,1,1,female,38.0,C,C
2,1,3,female,26.0,M,S
3,1,1,female,35.0,C,S
4,0,3,male,35.0,M,S
...,...,...,...,...,...,...
886,0,2,male,27.0,M,S
887,1,1,female,19.0,B,S
888,0,3,female,30.0,M,S
889,1,1,male,26.0,C,C


In [25]:
#Ahora realizamos OneHotEncoding para las columnas de sex, embarked y cabin

df_sex = pd.get_dummies(df['Sex'], prefix='Sex')
df_embarked = pd.get_dummies(df['Embarked'], prefix='Embarked')
df_cabin = pd.get_dummies(df['Cabin'], prefix='Cabin')

df = df.drop(['Sex', 'Embarked', 'Cabin'], axis=1)

df = pd.concat([df, df_sex, df_embarked, df_cabin], axis=1)

df

,Survived,Pclass,Age,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_M,Cabin_T
0,0,3,22.0,False,True,False,False,True,False,False,False,False,False,False,False,True,False
1,1,1,38.0,True,False,True,False,False,False,False,True,False,False,False,False,False,False
2,1,3,26.0,True,False,False,False,True,False,False,False,False,False,False,False,True,False
3,1,1,35.0,True,False,False,False,True,False,False,True,False,False,False,False,False,False
4,0,3,35.0,False,True,False,False,True,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,False,True,False,False,True,False,False,False,False,False,False,False,True,False
887,1,1,19.0,True,False,False,False,True,False,True,False,False,False,False,False,False,False
888,0,3,30.0,True,False,False,False,True,False,False,False,False,False,False,False,True,False
889,1,1,26.0,False,True,True,False,False,False,False,True,False,False,False,False,False,False


In [31]:
# Ahora crearemos el train/test

vectorizer = DictVectorizer(sparse=False)
df_x = df.drop(['Survived'], axis=1)
X = vectorizer.fit_transform(df_x.to_dict("records"))

y = df.Survived.values
classes = np.unique(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


El uso de One Hot Encoding es necesario para las variables categoricas del dataframe, dado que el algoritmo no entiende lo que significan las palabras, por lo que se cambian a numeros o a True/False para que pueda hacer calculos con ellos.

## Parte 3: Decision Tree (15 ptos.)
Entrena un Decision Tree y ajusta el parametro que controla la profundidad máxima del árbol, luego reporta las métricas vistas en el curso en los datos de testing.

In [32]:
range_T = [50, 100, 200, 500]

    
for t_ in range_T:
    clf = RandomForestClassifier(min_samples_split=50, n_estimators=t_, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    print(
        "For min_samples_split = 50 and t =",
        t_,
        "The accuracy is :",
        acc,
    )
    
for t_ in range_T:
    clf = RandomForestClassifier(min_samples_split=20, n_estimators=t_, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    print(
        "For min_samples_split = 20 and t =",
        t_,
        "The accuracy is :",
        acc,
    )
    
for t_ in range_T:
    clf = RandomForestClassifier(min_samples_split=10, n_estimators=t_, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    print(
        "For min_samples_split = 10 and t =",
        t_,
        "The accuracy is :",
        acc,
    )

    
for t_ in range_T:
    clf = RandomForestClassifier(min_samples_split=5, n_estimators=t_, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    print(
        "For min_samples_split = 5 and t =",
        t_,
        "The accuracy is :",
        acc,
    )

For min_samples_split = 50 and t = 50 The accuracy is : 0.7835820895522388
For min_samples_split = 50 and t = 100 The accuracy is : 0.8059701492537313
For min_samples_split = 50 and t = 200 The accuracy is : 0.8059701492537313
For min_samples_split = 50 and t = 500 The accuracy is : 0.8059701492537313
For min_samples_split = 20 and t = 50 The accuracy is : 0.7910447761194029
For min_samples_split = 20 and t = 100 The accuracy is : 0.7947761194029851
For min_samples_split = 20 and t = 200 The accuracy is : 0.7873134328358209
For min_samples_split = 20 and t = 500 The accuracy is : 0.7835820895522388
For min_samples_split = 10 and t = 50 The accuracy is : 0.7798507462686567
For min_samples_split = 10 and t = 100 The accuracy is : 0.7873134328358209
For min_samples_split = 10 and t = 200 The accuracy is : 0.7835820895522388
For min_samples_split = 10 and t = 500 The accuracy is : 0.7835820895522388
For min_samples_split = 5 and t = 50 The accuracy is : 0.7835820895522388
For min_samples_s

In [33]:
clf = RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

0.7798507462686567

In [34]:
from lime import lime_tabular

explainer = lime_tabular.LimeTabularExplainer(X_train, mode="classification", class_names=['Sano', 'Diabetes'],
                                              feature_names=x_df.columns,)


ModuleNotFoundError: No module named 'lime'

## Parte 4: Ensembles (30 ptos.)
Entrena los modelos Random Forest y AdaBoost, cada uno usando 50, 100 y 150 Decision Trees. Computa las métricas vistas en el curso y luego graficalas en el eje Y y la cantidad de árboles en el eje X. Compara los resultados con los obtenido en la parte 3.

Luego, ajusta la cantidad de árboles para cada uno de los ensembles, para esto apoyate de un gráfico error vs número de arboles.

### Preguntas:
- Explica en alto nivel, pero de forma completa cómo funcionan cada uno de los algoritmos.